# Extracting Data from zipfile

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%bash
apt-get update
apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

wget -q "https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz" > /dev/null
tar -xvf spark-3.1.1-bin-hadoop2.7.tgz > /dev/null

pip install pyspark findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 

In [3]:
# Set path to Spark
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

# Find Spark so that we can access session within our notebook
import findspark
findspark.init()

# Start SparkSession on all available cores
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import zipfile

fl = ["/content/drive/MyDrive/uspatentcitation.tsv.zip", "/content/drive/MyDrive/cpc_current.tsv.zip","/content/drive/MyDrive/cpc_subsection.tsv.zip"]#,"/content/drive/MyDrive/patent.tsv.zip"]
# f2 = ["/content/drive/MyDrive/patent.tsv.zip"]
directory_to_extract_to = '/content/drive/MyDrive/'

for i in fl:
  with zipfile.ZipFile(i, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall(directory_to_extract_to)
    print('Done!')

File Name                                             Modified             Size
uspatentcitation.tsv                           2021-03-06 14:45:20  11828933080
Extracting all the files now...
Done!
File Name                                             Modified             Size
cpc_current.tsv                                2021-03-17 14:34:12   4072664211
Extracting all the files now...
Done!
File Name                                             Modified             Size
cpc_subsection.tsv                             2021-03-03 09:01:26         8123
Extracting all the files now...
Done!


# Load Data to Datarame

In [4]:
data_cpc_subsection = spark.read.csv('/content/drive/MyDrive/cpc_subsection.tsv',
                      sep="\t",
                      header=True,
                      inferSchema=True)
data_cpc_subsection.printSchema()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)



In [5]:
from pyspark.sql.types import StructType,StructField,IntegerType,TimestampType,StringType


In [6]:
schema = StructType([
StructField('uuid', StringType(), True),
StructField('patent_id', StringType(), True),
StructField('section_id', StringType(), True),
StructField('subsection_id', StringType(), True),
StructField('group_id', StringType(), True),
StructField('subgroup_id', StringType(), True),
StructField('category', StringType(), True),
StructField('sequence', StringType(), True)
])
data_cpc_current = spark.read.csv('/content/drive/MyDrive/cpc_current.tsv',
                      sep="\t",
                      header=True,
                      schema=schema)
data_cpc_current.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- patent_id: string (nullable = true)
 |-- section_id: string (nullable = true)
 |-- subsection_id: string (nullable = true)
 |-- group_id: string (nullable = true)
 |-- subgroup_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sequence: string (nullable = true)



In [7]:
schema = StructType([
StructField('uuid', StringType(), True),
StructField('patent_id', StringType(), True),
StructField('citation_id', IntegerType(), True),
StructField('date', TimestampType(), True),
StructField('name', IntegerType(), True),
StructField('kind', StringType(), True),
StructField('country', StringType(), True),
StructField('category', StringType(), True),
StructField('sequence', StringType(), True)
])

data_uspatentcitation = spark.read.csv('/content/drive/MyDrive/uspatentcitation.tsv',
                      sep="\t",
                      header=True,
                      schema=schema)
data_uspatentcitation.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- patent_id: string (nullable = true)
 |-- citation_id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- name: integer (nullable = true)
 |-- kind: string (nullable = true)
 |-- country: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sequence: string (nullable = true)



In [ ]:
data_cpc_subsection.show(5)

+---+--------------------+
| id|               title|
+---+--------------------+
|A01|AGRICULTURE; FORE...|
|A21|BAKING; EDIBLE DO...|
|A22|BUTCHERING; MEAT ...|
|A23|FOODS OR FOODSTUF...|
|A24|TOBACCO; CIGARS; ...|
+---+--------------------+
only showing top 5 rows



In [ ]:
data_cpc_current.show(5)

+--------------------+---------+----------+-------------+--------+--------------+-----------+--------+
|                uuid|patent_id|section_id|subsection_id|group_id|   subgroup_id|   category|sequence|
+--------------------+---------+----------+-------------+--------+--------------+-----------+--------+
|00000146-5f8a-46c...|  5858193|         B|          B01|    B01J|B01J2219/00722| additional|      21|
|00000151-5e3f-498...|  6452431|         H|          H03|    H03L|     H03L7/107| additional|       4|
|00000252-cd5d-4f7...| 10107082|         G|          G01|    G01V| G01V2210/1234| additional|       7|
|0000026c-f4d2-4d5...|  8291562|         B|          B25|    B25B|     B25B27/00|inventional|       2|
|000002a2-cb75-41c...| 10289956|         H|          H03|    H03M|   H03M13/1117|inventional|       1|
+--------------------+---------+----------+-------------+--------+--------------+-----------+--------+
only showing top 5 rows



In [ ]:
data_uspatentcitation.show(5)

+--------------------+---------+-----------+-------------------+----+----+-------+------------------+--------+
|                uuid|patent_id|citation_id|               date|name|kind|country|          category|sequence|
+--------------------+---------+-----------+-------------------+----+----+-------+------------------+--------+
|00000jd7thmiucpao...|  5354551|    4875247|1989-10-01 00:00:00|null|null|     US|              null|      11|
|00000l0ooxrvfv6jk...|  D674253|       null|2006-04-01 00:00:00|null|   S|     US| cited by examiner|      13|
|00000nv59m7vgyvfx...|  D809697|    1963218|1934-06-01 00:00:00|null|   A|     US| cited by examiner|       0|
|00001jxaxme3a89hi...|  D808172|    7207426|2007-04-01 00:00:00|null|  B2|     US|cited by applicant|      14|
|00001nlwuimui60vu...|  8683318|    6642945|2003-11-01 00:00:00|null|  B1|     US| cited by examiner|       6|
+--------------------+---------+-----------+-------------------+----+----+-------+------------------+--------+
o

# Subsetting 

In [ ]:
Note that this section is different from 

We are interested in patents that cite citation, therefore we subset our data,i.e, `category` = 'cited by patent'

In [12]:
sub_cpc_current = data_cpc_current.select('patent_id','section_id', 'subsection_id')
sub_uspatentcitation = data_uspatentcitation.select('patent_id','citation_id', 'date')
# # sub_patent = data_patent.limit(5)

In [13]:
sub_uspatentcitation.printSchema()

root
 |-- patent_id: string (nullable = true)
 |-- citation_id: integer (nullable = true)
 |-- date: timestamp (nullable = true)



In [ ]:
sub_cpc_current.printSchema()

root
 |-- patent_id: string (nullable = true)
 |-- section_id: string (nullable = true)
 |-- subsection_id: string (nullable = true)



In [ ]:
##sub_uspatentcitation = sub_uspatentcitation.filter(sub_uspatentcitation.kind.contains('B'))#.limit(1000)

In [ ]:
sub_uspatentcitation.show(10)

+---------+-----------+-------------------+
|patent_id|citation_id|               date|
+---------+-----------+-------------------+
|  5354551|    4875247|1989-10-01 00:00:00|
|  D674253|       null|2006-04-01 00:00:00|
|  D809697|    1963218|1934-06-01 00:00:00|
|  D808172|    7207426|2007-04-01 00:00:00|
|  8683318|    6642945|2003-11-01 00:00:00|
| 10456544|    2317608|1943-04-01 00:00:00|
|  8250307|    7386701|2008-06-01 00:00:00|
|  D490798|       null|1961-06-01 00:00:00|
|  9199394|    5242647|1993-09-01 00:00:00|
| 10796471|    7648519|2010-01-01 00:00:00|
+---------+-----------+-------------------+
only showing top 10 rows



In [10]:
from pyspark.sql.functions import year, month, dayofmonth

In [15]:
sub_uspatentcitation = sub_uspatentcitation.withColumn('year', F.year(F.to_timestamp('date', 'yyyy/MM/dd')))
#sub_uspatentcitation = sub_uspatentcitation.withColumn('years',sub_uspatentcitation.select(year("date").alias('year')))

In [ ]:
sub_uspatentcitation.show(5)

+---------+-----------+-------------------+----+
|patent_id|citation_id|               date|year|
+---------+-----------+-------------------+----+
|  5354551|    4875247|1989-10-01 00:00:00|1989|
|  D674253|       null|2006-04-01 00:00:00|2006|
|  D809697|    1963218|1934-06-01 00:00:00|1934|
|  D808172|    7207426|2007-04-01 00:00:00|2007|
|  8683318|    6642945|2003-11-01 00:00:00|2003|
+---------+-----------+-------------------+----+
only showing top 5 rows



In [14]:
import pyspark.sql.functions as F

In [ ]:
# data_uspatentcitation = data_uspatentcitation.limit(10000)

In [ ]:
# data_uspatentcitation = data_uspatentcitation.filter(data_uspatentcitation.date > F.lit('2010-01-01 00:00:00'))

# Merging and Grouping 

Since one patent can have multiple associated `section`, `subsection`, `group` and `subgroup`, we decided to focus on the unique `subsection` information and ignore the duplicated `group` information.

In [ ]:
# unique_cpc_current = sub_cpc_current.select('patent_id','subsection_id')#,'category')

In [ ]:
del data_cpc_current  release some memory

In [16]:
# merged_patent = data_p_c.join(unique_cpc_current, data_p_c.patent_id == unique_cpc_current.patent_id,'left')
merged_patent = sub_uspatentcitation.join(sub_cpc_current, ['patent_id'],'left')

In [ ]:
merged_patent.printSchema()

root
 |-- patent_id: string (nullable = true)
 |-- citation_id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- section_id: string (nullable = true)
 |-- subsection_id: string (nullable = true)



In [17]:
citation_cpc = sub_cpc_current.withColumnRenamed('patent_id','citation_id').withColumnRenamed('subsection_id','citation_subsection_id').withColumnRenamed('section', 'citation_section')
                             
merged_pc = merged_patent.join(citation_cpc, ['citation_id'], 'left')

In [ ]:
merged_pc.printSchema()

root
 |-- citation_id: integer (nullable = true)
 |-- patent_id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- section_id: string (nullable = true)
 |-- subsection_id: string (nullable = true)
 |-- section_id: string (nullable = true)
 |-- citation_subsection_id: string (nullable = true)



In [18]:
merged_unique = merged_pc.dropDuplicates(['citation_id', 'patent_id'])

In [ ]:
# merged_unique.filter(merged_unique.citation_subsection_id.isNull()).count()

In [19]:
merged_dropna = merged_unique.filter(merged_unique.citation_subsection_id.isNotNull())

In [20]:
merged_dropna.printSchema()

root
 |-- citation_id: integer (nullable = true)
 |-- patent_id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- section_id: string (nullable = true)
 |-- subsection_id: string (nullable = true)
 |-- section_id: string (nullable = true)
 |-- citation_subsection_id: string (nullable = true)



In [21]:
new2 = merged_dropna.select('subsection_id','year', 'citation_subsection_id')

In [22]:
new3 = new2.groupBy(new2.subsection_id,new2.citation_subsection_id, new2.year).count()

In [ ]:
new3.count()

KeyboardInterrupt: ignored

In [ ]:
from pyspark.sql import SparkSession

In [24]:
newpd = new3.toPandas()
newpd

KeyboardInterrupt: ignored

# Plotting

In [23]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
#/content/drive/MyDrive/cpc_current.tsv
newpd.to_csv(r'/content/drive/MyDrive/new.csv', index = False, header=True)

NameError: ignored

In [ ]:
countheat = df.pivot(index='citation_subsection_id',columns='subsection_id',values='count')

In [ ]:
a4_dims = (10,8)
fig, ax = plt.subplots(figsize=a4_dims)
a = sns.heatmap(countheat, cmap = "Blues")
a